In [1]:
!pip install -q langchain langchain-community sentence-transformers chromadb transformers accelerate bitsandbytes pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.8/449.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 81.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q pymupdf tqdm pandas matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 59.2 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install -q sentence-transformers transformers accelerate torch

In [4]:
import os
import fitz
import torch
import chromadb
import pandas as pd
from tqdm.auto import tqdm
from chromadb.config import Settings
from langchain.schema import Document
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from sentence_transformers import SentenceTransformer
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

2025-11-07 10:00:19.031112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762509619.186239      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762509619.236109      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [6]:
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()
    return cleaned_text

def open_and_read_pdf(pdf_path: str, source_name: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, with source tracking.
    
    Returns:
        list[dict]: Pages with text and metadata (source document name)
    """
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    
    for page_number, page in tqdm(enumerate(doc), desc=f"Reading {source_name}"):
        text = page.get_text()  # Extract plain text (no images)
        text = text_formatter(text)
        
        pages_and_texts.append({
            "page_number": page_number,
            "source": source_name,  
            "page_char_count": len(text),
            "page_word_count": len(text.split(" ")),
            "page_sentence_count_raw": len(text.split(". ")),
            "page_token_count": len(text) / 4, 
            "text": text
        })
    
    return pages_and_texts


pdf_dir = '/kaggle/input/docaidataset'  

loader = DirectoryLoader(
    pdf_dir,
    glob="*.pdf",  
    loader_cls=PyPDFLoader, 
    show_progress=True,
    use_multithreading=True, 
    max_concurrency=4 
)

documents = loader.load()

print(f"✅ Loaded {len(documents)} documents")
print(f"Sample doc metadata: {documents[0].metadata if documents else 'No docs'}")

100%|██████████| 5/5 [00:03<00:00,  1.40it/s]

✅ Loaded 98 documents
Sample doc metadata: {'producer': 'iLovePDF', 'creator': 'PyPDF', 'creationdate': '', 'moddate': '2025-11-04T09:35:37+00:00', 'source': '/kaggle/input/docaidataset/react-21-33.pdf', 'total_pages': 13, 'page': 0, 'page_label': '1'}


In [7]:
all_pages_and_texts = []

for doc in documents:
    source_name = doc.metadata.get('source', 'Unknown').split('/')[-1]  
    page_number = doc.metadata.get('page', 0)
    text = text_formatter(doc.page_content) 
    
    all_pages_and_texts.append({
        "page_number": page_number,
        "source": source_name,
        "page_char_count": len(text),
        "page_word_count": len(text.split(" ")),
        "page_sentence_count_raw": len(text.split(". ")),
        "page_token_count": len(text) / 4,
        "text": text
    })

df = pd.DataFrame(all_pages_and_texts)
print(f"\nDataset Statistics by Source:")
print(df.groupby('source')[['page_char_count', 'page_word_count']].agg(['sum', 'mean']))



Dataset Statistics by Source:
                page_char_count              page_word_count            
                            sum         mean             sum        mean
source                                                                  
cot-1-22.pdf              77449  3520.409091           12437  565.318182
cot-23-43.pdf             58229  2772.809524           10428  496.571429
deepseekocr.pdf           53278  2421.727273            8427  383.045455
react-1-20.pdf            77085  3854.250000           10783  539.150000
react-21-33.pdf           32976  2536.615385            6000  461.538462


In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512, 
    chunk_overlap=100, 
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

text_chunks = text_splitter.split_documents(documents)
chunk_lengths = [len(chunk.page_content) for chunk in text_chunks]

print(f"Created {len(text_chunks)} chunks from {len(documents)} documents")
print(f"Chunk statistics:")
print(f" - Min: {min(chunk_lengths)} chars")
print(f" - Max: {max(chunk_lengths)} chars")
print(f" - Avg: {sum(chunk_lengths)/len(chunk_lengths):.0f} chars")

Created 762 chunks from 98 documents
Chunk statistics:
 - Min: 7 chars
 - Max: 512 chars
 - Avg: 454 chars


In [9]:
model_name = "Qwen/Qwen3-Embedding-0.6B"

embedding_model = SentenceTransformer(
    model_name,
    device=device,
    trust_remote_code=True
)

print(f"\n Embedding model loaded: {model_name}")
print(f"Embedding dimension: {embedding_model.get_sentence_embedding_dimension()}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]


 Embedding model loaded: Qwen/Qwen3-Embedding-0.6B
Embedding dimension: 1024


In [10]:
chunk_texts = [chunk.page_content for chunk in text_chunks]

print(f"\nGenerating embeddings for {len(chunk_texts)} chunks...")

embeddings = embedding_model.encode(
    chunk_texts,
    show_progress_bar=True,
    batch_size=16,
    normalize_embeddings=True, 
    convert_to_numpy=True
)

print(f" Embeddings generated! Shape: {embeddings.shape}")


Generating embeddings for 762 chunks...


Batches:   0%|          | 0/48 [00:00<?, ?it/s]

 Embeddings generated! Shape: (762, 1024)


In [11]:
client = chromadb.PersistentClient(path="/kaggle/working/chroma_db")
try:
    client.delete_collection(name="multi_doc_rag")
except:
    pass

collection = client.create_collection(
    name="multi_doc_rag",
    metadata={"description": "Multi-document RAG with source tracking"}
)

collection.add(
    embeddings=embeddings.tolist(),
    documents=chunk_texts,
    metadatas=[
        {
            "page_number": chunk.metadata.get("page_number", 0),  
            "source": chunk.metadata.get("source", "Unknown"),     
            "page": chunk.metadata.get("page", 0)
        } 
        for chunk in text_chunks
    ],
    ids=[f"chunk_{i}" for i in range(len(text_chunks))]
)

print(f"Added {collection.count()} chunks to vector store")

Added 762 chunks to vector store


In [12]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True, 
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=False 
)

model_name = "Qwen/Qwen2.5-7B"

print(f"Loading: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16 
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f" Model loaded! Size: {model.get_memory_footprint() / 1e9:.2f} GB")


Loading: Qwen/Qwen2.5-7B


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

 Model loaded! Size: 8.71 GB


In [20]:
conversation_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer",
    input_key="question",
    max_token_limit=2000  # Adjust based on your needs
)

In [21]:
def rewrite_question(question, memory):
    """
    Convert follow-up questions into standalone questions using LLM.
    """
    chat_history = memory.load_memory_variables({}).get("chat_history", [])
    
    if not chat_history or len(chat_history) == 0:
        return question
    
    history_text = ""
    for msg in chat_history[-4:]:  
        if isinstance(msg, HumanMessage):
            history_text += f"Q: {msg.content[:100]}\n"
        elif isinstance(msg, AIMessage):
            history_text += f"A: {msg.content[:100]}\n"
    
    rewrite_messages = [
        {
            "role": "system",
            "content": "Rewrite the follow-up question to be standalone. Output ONLY the rewritten question - no explanations."
        },
        {
            "role": "user",
            "content": f"""Context:
{history_text}
Current question: {question}

Rewritten standalone question:"""
        }
    ]
    
    prompt = tokenizer.apply_chat_template(
        rewrite_messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(model.device)
    input_length = inputs["input_ids"].shape[1]
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=30,  
            temperature=0.1,    # Deterministic
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=[tokenizer.eos_token_id, 151645],
        )
    
    response_tokens = outputs[0][input_length:]
    rewritten = tokenizer.decode(response_tokens, skip_special_tokens=True).strip()
    

    rewritten = rewritten.split('\n')[0].strip()
    

    if (len(rewritten) > 250 or 
        len(rewritten) < 5 or
        "assistant" in rewritten.lower() or
        "context:" in rewritten.lower() or
        rewritten.count("?") > 2):
        print(f" Rewrite validation failed, using original")
        return question
    
    print(f" Rewritten: {rewritten}")
    return rewritten


In [22]:
def query_rag_chatbot(question, memory=None, n_results=3, max_new_tokens=512, temperature=0.2, use_rewrite=True, similarity_threshold=0.15):
    """
    Query RAG with multi-turn conversation and hallucination prevention.
    
    Args:
        question: Current user question
        memory: ConversationBufferMemory object
        n_results: Number of retrieved chunks
        max_new_tokens: Max tokens to generate
        temperature: Sampling temperature
        use_rewrite: Whether to rewrite follow-up questions
        similarity_threshold: Minimum similarity score (default 0.15)
    
    Returns:
        answer: Generated answer
        sources: Retrieved source information
    """
    

    retrieval_question = question
    if use_rewrite and memory:
        chat_history = memory.load_memory_variables({}).get("chat_history", [])
        if len(chat_history) > 0:
            retrieval_question = rewrite_question(question, memory)
    

    query_embedding = embedding_model.encode(
        [retrieval_question], 
        normalize_embeddings=True,
        convert_to_numpy=True
    )
    

    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=n_results,
        include=['documents', 'metadatas', 'distances']
    )
    

    retrieved_docs = results['documents'][0]
    retrieved_metadata = results['metadatas'][0]
    distances = results['distances'][0]
    

    similarities = [1 - dist for dist in distances]
    

    relevant_results = [
        (doc, meta, sim) 
        for doc, meta, sim in zip(retrieved_docs, retrieved_metadata, similarities)
        if sim >= similarity_threshold
    ]
    

    if not relevant_results:
        no_context_answer = "I cannot find relevant information in the provided documents to answer this question. Please ask something related to the document content."
        
        if memory is not None:
            memory.save_context(
                {"question": question},
                {"answer": no_context_answer}
            )
        
        return no_context_answer, []
    

    filtered_docs = [item[0] for item in relevant_results]
    filtered_metadata = [item[1] for item in relevant_results]
    filtered_similarities = [item[2] for item in relevant_results]
    
    context = "\n\n".join([
        f"[Source: {meta['source']}, Page {meta.get('page', 0) + 1}]\n{doc}"
        for doc, meta in zip(filtered_docs, filtered_metadata)
    ])
    

    messages = [
        {
            "role": "system",
            "content": """You are a helpful AI assistant specialized in answering questions based strictly on provided context.

STRICT RULES:
1. Answer ONLY using information explicitly stated in the Context section
2. If the context doesn't contain the answer, respond with: "I cannot find this information in the provided documents."
3. DO NOT use your general knowledge or training data
4. DO NOT make up information, names, dates, or facts
5. When referencing information, cite the source document

If the question is outside the scope of the provided documents, politely decline to answer."""
        }
    ]
    

    if memory is not None:
        chat_history = memory.load_memory_variables({}).get("chat_history", [])
        
        for msg in chat_history[-6:]: 
            if isinstance(msg, HumanMessage):
                messages.append({"role": "user", "content": msg.content})
            elif isinstance(msg, AIMessage):
                messages.append({"role": "assistant", "content": msg.content})
    

    messages.append({
        "role": "user",
        "content": f"Context:\n{context}\n\nQuestion: {question}"
    })

    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    input_length = inputs["input_ids"].shape[1]
    
    stop_token_ids = [tokenizer.eos_token_id, 151645]
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            top_p=0.85,    
            do_sample=True,
            repetition_penalty=1.1, 
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=stop_token_ids
        )
    
    response_tokens = outputs[0][input_length:]
    answer = tokenizer.decode(response_tokens, skip_special_tokens=True)

    if memory is not None:
        memory.save_context(
            {"question": question},
            {"answer": answer}
        )
    

    sources = [
        {
            'source': meta.get('source', 'Unknown'),
            'page': meta.get('page', 0) + 1,
            'similarity': sim,
            'text_preview': doc[:100]
        }
        for doc, meta, sim in zip(filtered_docs, filtered_metadata, filtered_similarities)
    ]
    
    return answer, sources


In [23]:
# Reinitialize memory for clean start
conversation_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer",
    input_key="question"
)

print("🤖 Advanced Multi-turn RAG Chatbot with Hallucination Prevention")
print("Type 'quit' to exit, 'clear' to reset history\n" + "="*70)

while True:
    question = input("\n🧑 You: ").strip()
    
    if question.lower() in ['quit', 'exit', 'q']:
        print("👋 Goodbye!")
        break
    
    if question.lower() == 'clear':
        conversation_memory.clear()
        print("🗑️ [Conversation history cleared]")
        continue
    
    if not question:
        continue
    
    print("\n⏳ [Processing...]")
    
    answer, sources = query_rag_chatbot(
        question, 
        memory=conversation_memory,
        n_results=6,
        use_rewrite=True,
        similarity_threshold=0.15  # ADDED threshold parameter
    )
    
    print(f"\n🤖 Assistant: {answer}")
    

    if sources:
        print("\n📚 Sources:")
        for i, source in enumerate(sources, 1):
            quality = "🟢" if source['similarity'] > 0.3 else "🟡" if source['similarity'] > 0.15 else "🔴"
            print(f"{i}. {quality} {source['source']} (Page {source['page']}) - Relevance: {source['similarity']:.3f}")
            print(f"   Preview: {source['text_preview']}...")
    else:
        print("\n No relevant sources found - answer is based on rejection of irrelevant context")
    
    history_len = len(conversation_memory.load_memory_variables({}).get("chat_history", []))
    print(f"\n💬 [Chat history: {history_len//2} turn(s)]")
    print("="*70)


🤖 Advanced Multi-turn RAG Chatbot with Hallucination Prevention
Type 'quit' to exit, 'clear' to reset history



🧑 You:  what is chain of thought?



⏳ [Processing...]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🤖 Assistant: Chain-of-thought prompting involves providing a step-by-step thought process for arriving at the answer, mimicking how humans reason through problems. This method helps large language models perform complex arithmetic, commonsense, and symbolic reasoning tasks by breaking down multi-step problems into smaller parts. It's particularly useful because it encourages models to think sequentially and justify their answers, making them more reliable and interpretable.

📚 Sources:
1. 🟢 /kaggle/input/docaidataset/cot-1-22.pdf (Page 1) - Relevance: 0.345
   Preview: Figure 1: Chain-of-thought prompting enables large language models to tackle complex arithmetic,
com...
2. 🟡 /kaggle/input/docaidataset/cot-1-22.pdf (Page 3) - Relevance: 0.226
   Preview: mimics a step-by-step thought process for arriving at the answer (and also, solutions/explanations
t...
3. 🟡 /kaggle/input/docaidataset/cot-1-22.pdf (Page 3) - Relevance: 0.215
   Preview: language models can generate chains of though

...


⏳ [Processing...]
 Rewrite validation failed, using original


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


🤖 Assistant: ReAct is better than CoT in terms of being more factual and grounded while still maintaining accuracy in formulating reasoning structures. Additionally, ReAct incorporates elements from both ReAct and CoT-SC, allowing it to adaptively choose the appropriate method depending on whether it needs to rely on ReAct or CoT-SC. This flexibility enables ReAct to handle various types of success and failure modes effectively across different datasets like HotpotQA and Fever.

📚 Sources:
1. 🟢 /kaggle/input/docaidataset/react-1-20.pdf (Page 5) - Relevance: 0.491
   Preview: the problem solving process demonstrated by ReAct is more factual and grounded, whereas CoT
is more ...
2. 🟢 /kaggle/input/docaidataset/react-1-20.pdf (Page 6) - Relevance: 0.459
   Preview: Hallucination Hallucinated reasoning trace or facts 0% 56%
Label ambiguity Right prediction but did ...
3. 🟢 /kaggle/input/docaidataset/react-1-20.pdf (Page 8) - Relevance: 0.419
   Preview: comparing ReAct and Act can be foun


🧑 You:  exit


👋 Goodbye!
